In [2]:
import socket
import pickle
import struct
import cv2

In [3]:
try:
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    print("Socket successfully created")
except socket.error as err:
    print("Socket creation failed with error %s" % (err))


Socket successfully created


In [6]:

# Remove the extra space in the IP address and fix the port number
s.connect(('192.168.110.112', 9999))

data = b""
payload_size = struct.calcsize("L")  # Use "L" instead of "N" for unsigned long

while True:
    while len(data) < payload_size:
        packet = s.recv(4 * 1024)
        if not packet:
            break
        data += packet

    if not packet:
        break

    Pmsg_size = data[:payload_size]
    data = data[payload_size:]
    msg_size = struct.unpack("L", Pmsg_size)[0]

    while len(data) < msg_size:
        data += s.recv(4 * 1024)

    frame_data = data[:msg_size]
    data = data[msg_size:]
    frame = pickle.loads(frame_data)
    cv2.imshow("RECEIVING VIDEO", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

s.close()
cv2.destroyAllWindows()